In [1]:
import os
import numpy as np
import pandas as pd
import json
import tensorflow as tf
from datetime import datetime, timedelta

from dotenv import load_dotenv

from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape

import matplotlib.pyplot as plt

In [2]:
# Load .env file

load_dotenv()

True

In [3]:
# Set Alpaca API key and secret

ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_API_SECRET")
client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)

# Create the Alpaca API object

timeframe = TimeFrame(1, TimeFrameUnit.Day)
symbol = 'SPY'
start = datetime.utcnow() - timedelta(days=3650)
end=datetime.utcnow() - timedelta(days=730)
request = StockBarsRequest(symbol_or_symbols=symbol, start=start, end=end, timeframe=timeframe)

In [4]:
source_df = client.get_stock_bars(request).df.tz_convert('America/New_York', level=1)

In [5]:
display(source_df.head())
display(source_df.tail())

open    high     low     close  \
symbol timestamp                                                      
SPY    2016-01-04 00:00:00-05:00  200.490  201.03  198.59  201.0192   
       2016-01-05 00:00:00-05:00  201.405  201.90  200.05  201.3600   
       2016-01-06 00:00:00-05:00  198.330  200.06  197.60  198.8200   
       2016-01-07 00:00:00-05:00  195.330  197.44  193.59  194.0500   
       2016-01-08 00:00:00-05:00  195.190  195.85  191.58  191.9230   

                                       volume  trade_count        vwap  
symbol timestamp                                                        
SPY    2016-01-04 00:00:00-05:00  225903783.0     655489.0  200.656423  
       2016-01-05 00:00:00-05:00  112719152.0     418709.0  201.084280  
       2016-01-06 00:00:00-05:00  153948196.0     548386.0  198.956460  
       2016-01-07 00:00:00-05:00  216191953.0     796451.0  195.345911  
       2016-01-08 00:00:00-05:00  216105404.0     754102.0  193.644537

open    high     low   close       volume  \
symbol timestamp                                                                
SPY    2022-02-18 00:00:00-05:00  437.33  438.66  431.82  434.23  132619100.0   
       2022-02-22 00:00:00-05:00  431.89  435.50  425.86  429.57  126971668.0   
       2022-02-23 00:00:00-05:00  432.66  433.26  421.35  422.07  132864067.0   
       2022-02-24 00:00:00-05:00  411.02  428.76  410.64  427.99  220033458.0   
       2022-02-25 00:00:00-05:00  429.61  437.84  427.86  437.75  123640399.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2022-02-18 00:00:00-05:00    1134344.0  435.238076  
       2022-02-22 00:00:00-05:00    1213515.0  430.690333  
       2022-02-23 00:00:00-05:00    1166042.0  425.926191  
       2022-02-24 00:00:00-05:00    2316988.0  419.890520  
       2022-02-25 00:00:00-05:00    1140051.0  434.397245

In [6]:
bars_df = source_df.copy()
# pct_change is profit from last close
bars_df["pct_change"] = bars_df["close"].pct_change()
# signal for when we want to be in or out of a stock
#bars_df["signal"] = np.where(bars_df["pct_change"] > 0, 1.0, 0.0)
# reaction is the signal diff
#bars_df["reaction"] = bars_df["signal"].diff()
# action is if we could perfectly predict the next close
#bars_df["action"] = bars_df["reaction"].shift(-1)
# these values are the high, low, and open as a percentage of the current close
bars_df["high %"] = (bars_df["high"] - bars_df["close"])/bars_df["close"]
bars_df["low %"] = (bars_df["low"] - bars_df["close"])/bars_df["close"]
bars_df["open %"] = (bars_df["open"] - bars_df["close"])/bars_df["close"]

In [7]:
bars_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1549 entries, ('SPY', Timestamp('2016-01-04 00:00:00-0500', tz='America/New_York')) to ('SPY', Timestamp('2022-02-25 00:00:00-0500', tz='America/New_York'))
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         1549 non-null   float64
 1   high         1549 non-null   float64
 2   low          1549 non-null   float64
 3   close        1549 non-null   float64
 4   volume       1549 non-null   float64
 5   trade_count  1549 non-null   float64
 6   vwap         1549 non-null   float64
 7   pct_change   1548 non-null   float64
 8   high %       1549 non-null   float64
 9   low %        1549 non-null   float64
 10  open %       1549 non-null   float64
dtypes: float64(11)
memory usage: 182.3+ KB


In [8]:
# Cleanup DF for model

bars_df = bars_df.droplevel(level=0).dropna()
bars_df.head()

,open,high,low,close,volume,trade_count,vwap,pct_change,high %,low %,open %
timestamp,,,,,,,,,,,
2016-01-05 00:00:00-05:00,201.405,201.90,200.05,201.360,112719152.0,418709.0,201.084280,0.001695,0.002682,-0.006506,0.000223
2016-01-06 00:00:00-05:00,198.330,200.06,197.60,198.820,153948196.0,548386.0,198.956460,-0.012614,0.006237,-0.006136,-0.002465
2016-01-07 00:00:00-05:00,195.330,197.44,193.59,194.050,216191953.0,796451.0,195.345911,-0.023992,0.017470,-0.002371,0.006596
2016-01-08 00:00:00-05:00,195.190,195.85,191.58,191.923,216105404.0,754102.0,193.644537,-0.010961,0.020461,-0.001787,0.017022
2016-01-11 00:00:00-05:00,193.010,193.41,189.82,192.110,205368067.0,701548.0,191.757659,0.000974,0.006767,-0.011920,0.004685


In [9]:
# Normalize data set

scaler = StandardScaler()
bars_df_scaled = scaler.fit_transform(bars_df)


In [10]:
# Define constants

# input shape

input_shape = bars_df_scaled.shape[1]
latent_dim = 11
num_samples = bars_df_scaled.shape[0]
batch_size = 16


In [11]:
# Define generator (Sequential)

build_generator = Sequential([
    Dense(128,input_shape=(input_shape,), activation="relu"),
    Dense(256, activation="relu"),
    Dense(512, activation="relu"),
    Dense(input_shape, activation="sigmoid")
])

# Define descriminator

build_discriminator = Sequential([
    Dense(512, input_shape=(input_shape,), activation="relu"),
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])

2024-02-25 17:23:32.984470: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-02-25 17:23:32.984491: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-02-25 17:23:32.984497: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-02-25 17:23:32.984527: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-25 17:23:32.984545: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
# Compile generator
build_generator.compile(loss = "mse", optimizer="adam")

# Compile discriminator
build_discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Combine models

# Confirm that discriminator weights are not training during generators training

build_discriminator.trainable=False

# Setup for generators data

z = tf.keras.Input(shape=(latent_dim,))

# Generated data by the generator

generated_data = build_generator(z)

# Discriminators verdict

validity = build_discriminator(generated_data)

In [14]:
# Define combined models

combined = tf.keras.Model(z, validity)
combined.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

# Print summary of the combined model
combined.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 11)]              0         
                                                                 
 sequential (Sequential)     (None, 11)                171787    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 170497    
                                                                 
Total params: 342284 (1.31 MB)
Trainable params: 171787 (671.04 KB)
Non-trainable params: 170497 (666.00 KB)
_________________________________________________________________


In [15]:
# Define training loop

epochs = 10000

for epoch in range(epochs):
    # Train discriminator
    # Sample real data

    idx = np.random.choice(num_samples, batch_size, replace=False)
    real_data = bars_df_scaled[idx]

    # Generate fake data (Random Walk noise)

    gaussian_noise = np.random.normal(0,1,size=(batch_size,input_shape))
    random_walk_noise = np.cumsum(gaussian_noise, axis=0)

    fake_data = build_generator.predict(random_walk_noise)

    # Train discriminator

    d_loss_real = build_discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
    d_loss_fake = build_discriminator.train_on_batch(fake_data, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train generator 

    noise = np.random.normal(0, 1, (batch_size, input_shape))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
    
    # Print progress

    print(f"Epoch {epoch}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")


1/1 [==============================] - 0s 67ms/step


2024-02-25 17:23:33.389516: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-25 17:23:33.581032: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-25 17:23:33.990946: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 0, Discriminator Loss: [0.7866348 0.15625  ], Generator Loss: [0.3020637333393097, 0.0]
1/1 [==============================] - 0s 13ms/step
Epoch 1, Discriminator Loss: [0.58048677 0.75      ], Generator Loss: [0.46942365169525146, 0.0]
1/1 [==============================] - 0s 12ms/step
Epoch 2, Discriminator Loss: [0.44936982 0.875     ], Generator Loss: [0.6536977291107178, 0.0]
1/1 [==============================] - 0s 11ms/step
Epoch 3, Discriminator Loss: [0.57481403 0.8125    ], Generator Loss: [0.7253929376602173, 0.0]
1/1 [==============================] - 0s 12ms/step
Epoch 4, Discriminator Loss: [0.25795402 0.90625   ], Generator Loss: [0.8212270140647888, 0.0]
1/1 [==============================] - 0s 12ms/step
Epoch 5, Discriminator Loss: [0.65377861 0.84375   ], Generator Loss: [0.8325440883636475, 0.0]
1/1 [==============================] - 0s 12ms/step
Epoch 6, Discriminator Loss: [0.69154479 0.8125    ], Generator Loss: [0.799293577671051, 0.0]
1/1 [=============

In [16]:
# Saving Model Weights

build_generator.save("generator_model.h5")
build_discriminator.save("discriminator_model.h5")

/Users/ebrown/anaconda3/envs/GAN_kernel/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
# Create alpaca object for testing

timeframe = TimeFrame(1, TimeFrameUnit.Day)
symbol = 'SPY'
start = datetime.utcnow() - timedelta(days=730)
end=datetime.utcnow() - timedelta(days=1)
request = StockBarsRequest(symbol_or_symbols=symbol, start=start, end=end, timeframe=timeframe)

In [18]:
# Create df for testing

test_df = client.get_stock_bars(request).df.tz_convert('America/New_York', level=1)

# data preprocessing for testing

# pct_change is profit from last close
test_df["pct_change"] = test_df["close"].pct_change()
# signal for when we want to be in or out of a stock
#bars_df["signal"] = np.where(bars_df["pct_change"] > 0, 1.0, 0.0)
# reaction is the signal diff
#bars_df["reaction"] = bars_df["signal"].diff()
# action is if we could perfectly predict the next close
#bars_df["action"] = bars_df["reaction"].shift(-1)
# these values are the high, low, and open as a percentage of the current close
test_df["high %"] = (test_df["high"] - test_df["close"])/test_df["close"]
test_df["low %"] = (test_df["low"] - test_df["close"])/test_df["close"]
test_df["open %"] = (test_df["open"] - test_df["close"])/test_df["close"]

# set timestamp as index, drop nan

test_df = test_df.droplevel(level=0).dropna()

test_df.info()
display(test_df.head())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 499 entries, 2022-03-01 00:00:00-05:00 to 2024-02-23 00:00:00-05:00
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         499 non-null    float64
 1   high         499 non-null    float64
 2   low          499 non-null    float64
 3   close        499 non-null    float64
 4   volume       499 non-null    float64
 5   trade_count  499 non-null    float64
 6   vwap         499 non-null    float64
 7   pct_change   499 non-null    float64
 8   high %       499 non-null    float64
 9   low %        499 non-null    float64
 10  open %       499 non-null    float64
dtypes: float64(11)
memory usage: 46.8 KB


,open,high,low,close,volume,trade_count,vwap,pct_change,high %,low %,open %
timestamp,,,,,,,,,,,
2022-03-01 00:00:00-05:00,435.04,437.1700,427.11,429.98,139991559.0,1422070.0,431.644816,-0.015230,0.016722,-0.006675,0.011768
2022-03-02 00:00:00-05:00,432.37,439.7200,431.57,437.89,120234911.0,1252759.0,436.251696,0.018396,0.004179,-0.014433,-0.012606
2022-03-03 00:00:00-05:00,440.47,441.1100,433.80,435.71,105501865.0,1039162.0,436.908456,-0.004978,0.012394,-0.004384,0.010925
2022-03-04 00:00:00-05:00,431.75,433.3700,427.88,432.17,113974262.0,1083494.0,430.920534,-0.008125,0.002777,-0.009927,-0.000972
2022-03-07 00:00:00-05:00,431.55,432.3018,419.36,419.43,138047941.0,1448623.0,423.968129,-0.029479,0.030689,-0.000167,0.028896


In [19]:
# Normalize test data set

test_df_scaled = scaler.fit_transform(test_df)

print(test_df_scaled)
print(len(test_df_scaled))

[[ 0.44939782  0.43657467  0.28426178 ...  1.4931797   0.08293824
   1.25569624]
 [ 0.36620036  0.5174457   0.42062882 ... -0.31872284 -1.15281987
  -1.26945759]
 [ 0.61859714  0.56152834  0.48881235 ...  0.86793541  0.44787461
   1.1683306 ]
 ...
 [ 2.33084439  2.34576518  2.3160084  ... -0.87595423 -0.02318329
  -0.33644817]
 [ 2.59850961  2.69842629  2.60525329 ... -0.64063682 -0.2599813
  -0.67592284]
 [ 2.76241172  2.75043746  2.73000161 ... -0.27388488  0.91089219
   0.3262019 ]]
499


In [36]:
# Test generator against real data

#lists to store generated and timestamps

column_names=[
    "Predicted Open",
    "Predicted High",
    "Predicted Low",
    "Predicted Close",
    "Predicted Volume",
    "Predicted Trade Count",
    "Predicted VWAP",
    "Predicted pct_change",
    "Predicted high %",
    "Predicted low %",
    "Predicted open %"
    ]

# sliding window

window_size = 1

# init lists to store generated data with associated timestamps

timestamps = []
predictions = []

In [37]:
# shape of scaled test data

print(test_df_scaled.shape)

(499, 11)


In [38]:
predictions_df = pd.DataFrame(columns = column_names)

predictions_df[:] = 0
predictions_df.head()



,Predicted Open,Predicted High,Predicted Low,Predicted Close,Predicted Volume,Predicted Trade Count,Predicted VWAP,Predicted pct_change,Predicted high %,Predicted low %,Predicted open %


In [46]:

for i in range(len(test_df_scaled) - window_size):

    # extract sliding window of real data

    input_data = test_df_scaled[i:i+window_size]

    # predict the next row using generator

    #predicted_row = build_generator.predict(input_data.reshape(1, window_size, 11))
    predicted_row = build_generator.predict(input_data)

    #append generated row to list
    # predictions.append(predicted_row)

    # extract timestamp for the prediction

    timestamp = test_df.index[i+window_size]
    
    # predictions_df.loc[timestamp] = predicted_row.squeeze()

    predictions_df = pd.concat([predictions_df,predicted_row],ignore_index=True)


1/1 [==============================] - 0s 15ms/step


TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [42]:
print(predictions)

[]


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 494 has 0 dimension(s)

In [ ]:

predictions_array = np.array(predictions)

reshaped_predictions = predictions_array.reshape(5,11)

predictions_df = pd.DataFrame(reshaped_predictions)

/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_25835/4240117929.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predictions_array = np.array(predictions)


ValueError: cannot reshape array of size 1976 into shape (5,10)

In [ ]:
print("original df shape", test_df_scaled.shape)
print("generated df shape", generated_data.shape)

print("Original dataframe summary statistics:")
print((test_df_scaled))

original df shape (499, 11)
generated df shape (None, 11)
Original dataframe summary statistics:
[[ 0.44939782  0.43657467  0.28426178 ...  1.4931797   0.08293824
   1.25569624]
 [ 0.36620036  0.5174457   0.42062882 ... -0.31872284 -1.15281987
  -1.26945759]
 [ 0.61859714  0.56152834  0.48881235 ...  0.86793541  0.44787461
   1.1683306 ]
 ...
 [ 2.33084439  2.34576518  2.3160084  ... -0.87595423 -0.02318329
  -0.33644817]
 [ 2.59850961  2.69842629  2.60525329 ... -0.64063682 -0.2599813
  -0.67592284]
 [ 2.76241172  2.75043746  2.73000161 ... -0.27388488  0.91089219
   0.3262019 ]]


In [ ]:
generated_data_inverse = scaler.inverse_transform(generated_test_df)
generated_data_df = pd.DataFrame(generated_data_inverse)
generated_data_df.head(22)

,0,1,2,3,4,5,6,7,8,9,10
0,420.617767,423.404053,417.812988,420.793365,109802704.0,840040.3750,420.666901,0.000381,0.013308,-0.006875,0.009300
1,420.617767,453.926910,417.813202,453.008636,85364960.0,832540.2500,420.666901,0.000375,0.010957,-0.007195,-0.000353
2,420.617798,423.404114,418.422791,420.793365,109802424.0,840040.3750,420.740021,0.000393,0.013308,-0.007085,0.009300
3,420.617767,423.404053,417.812988,420.793365,109802704.0,840040.3750,420.666901,0.012361,0.013308,-0.000917,0.009300
4,452.710114,454.935760,450.518829,453.008636,85364928.0,634421.8125,452.755493,0.000375,0.006385,-0.007195,-0.000353
5,420.617767,453.926910,417.813202,453.008636,85364960.0,832540.2500,420.666901,0.000375,0.010957,-0.007195,-0.000353
6,420.617798,423.404114,418.422791,420.793365,109802424.0,840040.3750,420.740021,0.000393,0.013308,-0.007085,0.009300
7,420.617767,423.404053,417.812988,420.793365,109802704.0,840040.3750,420.666901,0.012361,0.013308,-0.000917,0.009300
8,452.710114,454.935760,450.518829,453.008636,85364928.0,634421.8125,452.755493,0.000375,0.006385,-0.007195,-0.000353
9,452.710114,454.935760,450.518829,453.008636,85364928.0,622784.7500,452.755493,0.000375,0.006385,-0.007195,-0.000353


In [ ]:
test_df.head(22)

,open,high,low,close,volume,trade_count,vwap,pct_change,high %,low %,open %
timestamp,,,,,,,,,,,
2022-03-01 00:00:00-05:00,435.04,437.1700,427.110,429.98,139991559.0,1422070.0,431.644816,-0.015230,0.016722,-0.006675,0.011768
2022-03-02 00:00:00-05:00,432.37,439.7200,431.570,437.89,120234911.0,1252759.0,436.251696,0.018396,0.004179,-0.014433,-0.012606
2022-03-03 00:00:00-05:00,440.47,441.1100,433.800,435.71,105501865.0,1039162.0,436.908456,-0.004978,0.012394,-0.004384,0.010925
2022-03-04 00:00:00-05:00,431.75,433.3700,427.880,432.17,113974262.0,1083494.0,430.920534,-0.008125,0.002777,-0.009927,-0.000972
2022-03-07 00:00:00-05:00,431.55,432.3018,419.360,419.43,138047941.0,1448623.0,423.968129,-0.029479,0.030689,-0.000167,0.028896
2022-03-08 00:00:00-05:00,419.62,427.2100,415.120,416.25,167638902.0,1950904.0,419.707824,-0.007582,0.026330,-0.002715,0.008096
2022-03-09 00:00:00-05:00,425.14,429.5100,422.820,427.41,117271076.0,1133573.0,426.179666,0.026811,0.004913,-0.010739,-0.005311
2022-03-10 00:00:00-05:00,422.52,426.4300,420.440,425.48,95329689.0,917460.0,423.894551,-0.004516,0.002233,-0.011845,-0.006957
2022-03-11 00:00:00-05:00,428.12,428.7700,419.530,420.07,95687775.0,835159.0,423.963271,-0.012715,0.020711,-0.001286,0.019163


,0,1,2,3,4,5,6,7,8,9,10
0,0.007778,0.035432,-0.062695,-0.118824,0.422381,1.245202,-0.047234,-0.641091,1.695312,0.685944,1.004966
1,-0.027152,0.108114,0.017787,0.166040,0.925271,1.483095,0.066484,1.805528,-0.695106,-1.748430,-1.517021
2,0.288464,0.388686,0.213024,0.459602,2.419187,3.563780,0.333686,1.820880,-0.902494,-2.808112,-1.323950
3,0.403858,0.559758,0.469770,0.628905,0.755506,0.859020,0.531743,1.013933,-0.922408,-1.697526,-1.718117
4,0.541394,0.680049,0.593094,0.736078,0.856417,0.861511,0.649017,0.622372,-0.811822,-1.467614,-1.480767
5,0.739122,0.730831,0.698975,0.732040,0.136152,1.427573,0.720698,-0.055225,-0.248938,-0.181633,0.026974
6,0.786526,0.861595,0.857491,0.893577,-0.348319,0.058989,0.869902,0.946337,-0.604039,-0.173339,-0.820794
7,0.819273,0.795261,0.791697,0.713712,-0.169919,0.079794,0.789482,-1.106227,0.605502,1.115823,0.764548
8,0.789021,0.859056,0.823829,0.921535,-0.837096,-0.389746,0.842638,1.227997,-0.919199,-0.877842,-1.007634
9,0.951819,0.937769,0.936136,0.989877,-0.305228,0.168751,0.946972,0.377004,-0.829787,-0.350772,-0.311481


In [ ]:
# Create alpaca object for testing

timeframe = TimeFrame(1, TimeFrameUnit.Day)
symbol = 'SPY'
start = datetime.utcnow() - timedelta(days=3)
end=datetime.utcnow() - timedelta(days=1)
request = StockBarsRequest(symbol_or_symbols=symbol, start=start, end=end, timeframe=timeframe)

current_df = client.get_stock_bars(request).df.tz_convert('America/New_York', level=1)
current_df.head()

open    high     low   close      volume  \
symbol timestamp                                                               
SPY    2024-02-22 00:00:00-05:00  504.01  508.49  503.02  507.50  76402535.0   
       2024-02-23 00:00:00-05:00  509.27  510.13  507.10  507.85  61309016.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2024-02-22 00:00:00-05:00     613961.0  505.832024  
       2024-02-23 00:00:00-05:00     489047.0  508.438618

In [ ]:
# # Define constants

# latent_dim = 100
# input_shape = bars_df.shape[1]
# output_shape = bars_df.shape[1]
# num_samples = 1000
# gaussian_noise = np.random.normal(0,1,size=(num_samples, latent_dim))
# random_walk_noise = np.cumsum(gaussian_noise,axis=0)

In [ ]:
# # define model building functions

# def build_generator(num_layers,layer_size,activation,output_activation):
#     model = models.Sequential()
#     model.add(layers.Dense(layer_size, activation = activation, input_dim=num_layers))

#     for _ in range(num_layers-1):
#         model.add(layers.Dense(layer_size, activation = activation))

#     model.add(layers.Dense(output_shape,activation = output_activation))

#     pass

# # def build_discriminator(layers,layer_size,activation):

# def build_discriminator(num_layers,layer_size,activation):
#     model = models.Sequential()
#     model.add(layers.Dense(layer_size, activation = activation, input_dim = input_shape))

#     for _ in range(num_layers - 1):
#         model.add(layers.Dense(layer_size, activation = activation))

#     model.add(layers.Dense(1))

#     pass